In [ ]:
import json
import numpy as np

# Your JSON data
json_file_path = "./environment_maincrops/data/maincrops.json"

# Open and read the JSON file
with open(json_file_path, "r") as json_file:
    # Parse the JSON data into a Python dictionary
    data = json.load(json_file)

# # Function to calculate average and standard deviation
def calculate_stats(values):
    avg = np.mean(values)
    std = np.std(values)
    return avg, std

# # Iterate through each crop
for crop_key, crop_data in data.items():
    ertrag_data = crop_data["Ertrag"]
    
    # Iterate through each region under "Ertrag"
    for region_key, region_data in ertrag_data.items():
        values = region_data["Values"]
        
        # Calculate average and standard deviation
        avg, std = calculate_stats(values)
        
        # Add "avg" and "std" to the region_data
        region_data["avg"] = avg
        region_data["std"] = std

# # Convert the updated data back to JSON format
updated_json = json.dumps(data, indent=2)

# # Print the updated JSON data
print(updated_json)

In [16]:
#### Append GBM parameters


import numpy as np
import json
import numpy as np

# Your JSON data
json_file_path = "./environment_maincrops/data/maincrops.json"

# Open and read the JSON file
with open(json_file_path, "r") as json_file:
    # Parse the JSON data into a Python dictionary
    data = json.load(json_file)

# # Function to calculate average and standard deviation
def calculate_stats(values):
    avg = np.mean(values)
    std = np.std(values)
    return avg, std


# # # Function to calculate average and standard deviation
def calculate_stats_gbm(values):
    avg = np.mean(values)
    std = np.std(values)

    returns = np.diff(values) / values[:-1]

    # Calculate the average annual return (drift)
    avg_return = np.mean(returns)

    # Calculate annual volatility (sigma)
    volatility = np.std(returns)
    return avg, std, avg_return, volatility



kosten_keys = ["Saatgut","Dünger N","Dünger P","Dünger K","Sonstiges"]
# # # Iterate through each crop
for crop_key, crop_data in data.items():
    print(crop_key)

    # Ertrag-Stats
    ertrag_data = crop_data["Ertrag"]
    
    # Iterate through each region under "Ertrag"
    for region_key, region_data in ertrag_data.items():
        values = region_data["Values"]
        
        # Calculate average and standard deviation
        avg, std = calculate_stats(values)
        
        # Add "avg" and "std" to the region_data
        region_data["avg"] = avg
        region_data["std"] = std

    # Verkaufspreis GBM Stats
    verkaufspreis = crop_data["Verkaufspreis"]
    values = region_data["Values"]
        
    # Calculate average and standard deviation
    avg, std, avg_return, volatility = calculate_stats_gbm(values)
        
    # Add "avg" and "std" to the region_data
    verkaufspreis["avg"] = avg
    verkaufspreis["std"] = std
    verkaufspreis["avg_gbm"] = avg_return
    verkaufspreis["std_gbm"] = volatility


    # Kosten GBM Stats
    kosten = crop_data["Kosten"]
    for key in kosten_keys:
        values = kosten[key]["Values"]
        avg, std, avg_return, volatility = calculate_stats_gbm(values)
        kosten[key]["avg"] = avg
        kosten[key]["std"] = std
        kosten[key]["avg_gbm"] = avg_return
        kosten[key]["std_gbm"] = volatility

# Dump the dictionary to a JSON file with ensure_ascii=False
with open("./environment_maincrops/data/maincrops_updated.json", "w", encoding="utf-8") as json_file:
    json.dump(data, json_file, ensure_ascii=False)

KLEEGRAS
LUZERNE
ACKERBOHNE
KÖRNERERBSE
SOJABOHNE
FUTTERLUPINE
WINTERWEIZEN
SOMMERWEIZEN
WINTERHARTWEIZEN
WINTERDINKEL
WINTERTRITICALE
WINTERROGGEN
WINTERGERSTE
SOMMERGERSTE
SOMMERHAFER
HIRSE
SILOMAIS
KÖRNERMAIS
ZUCKERRÜBEN
KARTOFFELN
WINTERRAPS
SONNENBLUMEN
ÖLKÜRBIS


### Kolbe matrix as dictionary

In [38]:
import pandas as pd
import pprint

class CustomPrettyPrinter(pprint.PrettyPrinter):
    def format(self, object, context, maxlevels, level):
        if isinstance(object, list):
            # Customize how lists are formatted
            if len(object) > 5:  # Adjust this threshold as needed
                return self._format_multiline_list(object, context, maxlevels, level)
            else:
                return super().format(object, context, maxlevels, level)
        return super().format(object, context, maxlevels, level)

    def _format_multiline_list(self, object, context, maxlevels, level):
        output = "[" + ", ".join(map(repr, object)) + "]"
        return output, True, False

pp = CustomPrettyPrinter(width=160)
# Read the Excel file into a DataFrame
excel_file = './environment_maincrops/data/kolbe.xlsx'  # Replace with the path to your Excel file
df = pd.read_excel(excel_file, header=1, index_col=0)

# Create a dictionary to store the matrix
matrix_dict = {"PRECROP": {}}

# Map plant names to integers
plant_mapping = {
    'KLEEGRAS': 0,
    'LUZERNE': 1,
    'ACKERBOHNE': 2,
    'KÖRNERERBSE': 3,
    'SOJABOHNE': 4,
    'FUTTERLUPINE': 5,
    'WINTERWEIZEN': 6,
    'SOMMERWEIZEN': 7,
    'WINTERHARTWEIZEN': 8,
    'WINTERDINKEL': 9,
    'WINTERTRITICALE': 10,
    'WINTERROGGEN': 11,
    'WINTERGERSTE': 12,
    'SOMMERGERSTE': 13,
    'SOMMERHAFER': 14,
    'HIRSE': 15,
    'SILOMAIS': 16,
    'KÖRNERMAIS': 17,
    'ZUCKERRÜBEN': 18,
    'KARTOFFELN': 19,
    'WINTERRAPS': 20,
    'SONNENBLUMEN': 21,
    'ÖLKÜRBIS': 22
}

# Iterate through the rows of the DataFrame and populate the matrix_dict
for row_name, row_values in df.iterrows():
    row_index = plant_mapping.get(row_name)
    matrix_dict["PRECROP"][row_index] = []
    first_col = True
    for col_name, col_value in row_values.items():
        if first_col:
            first_col = False
            continue
        col_index = plant_mapping.get(col_name)
        matrix_dict["PRECROP"][row_index].append(col_value)
    if row_name == "ÖLKÜRBIS":
        break

# Print or use the matrix_dict as needed
pp.pprint(matrix_dict)

{'PRECROP': {0: [-2.0, -2.0, -2.0, -2.0, -2.0, -2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 2.0, 2.0, 1.0, 2.0, 2.0, -2.0, -1.0],
             1: [-2.0, -2.0, -2.0, -2.0, -2.0, -2.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 2.0, 2.0, 1.0, 1.0, 1.0, -1.0, -1.0],
             2: [-2.0, -2.0, -2.0, -2.0, -2.0, -2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, -2.0, -1.0, 2.0],
             3: [-2.0, -2.0, -2.0, -2.0, -2.0, -2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 1.0, 1.0, 1.0, -1.0, 2.0],
             4: [-2.0, -2.0, -2.0, -2.0, -2.0, -2.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, -2.0, -1.0, -1.0],
             5: [-2.0, -2.0, -2.0, -2.0, -2.0, -2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, -2.0, -1.0, 2.0],
             6: [1.0, 2.0, -1.0, -1.0, 2.0, -1.0, -2.0, -2.0, -2.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, -1.0, 1.0, 2.0],
    

### Create dictionary for crop breaks

In [59]:
import pprint
import pandas

plant_mapping = {
    'KLEEGRAS': 0,
    'LUZERNE': 1,
    'ACKERBOHNE': 2,
    'KÖRNERERBSE': 3,
    'SOJABOHNE': 4,
    'FUTTERLUPINE': 5,
    'WINTERWEIZEN': 6,
    'SOMMERWEIZEN': 7,
    'WINTERHARTWEIZEN': 8,
    'WINTERDINKEL': 9,
    'WINTERTRITICALE': 10,
    'WINTERROGGEN': 11,
    'WINTERGERSTE': 12,
    'SOMMERGERSTE': 13,
    'SOMMERHAFER': 14,
    'HIRSE': 15,
    'SILOMAIS': 16,
    'KÖRNERMAIS': 17,
    'ZUCKERRÜBEN': 18,
    'KARTOFFELN': 19,
    'WINTERRAPS': 20,
    'SONNENBLUMEN': 21,
    'ÖLKÜRBIS': 22
}


pp = pprint.PrettyPrinter(indent=2)

# Create the dictionary
result = {}

excel_file = './environment_maincrops/data/kolbe.xlsx'  # Replace with the path to your Excel file
df = pd.read_excel(excel_file,sheet_name = 1, header=0, index_col=1)

# Iterate through rows
for index, row in df.iterrows():
    if pd.isna(index):
        break
    # print(plant_mapping[index])
    crop_name = index
    ap_groups = []
    mf_groups = []

    # Iterate through columns starting with 'AP' or 'MF'
    for column_name, value in row.items():
        if column_name in ["Familie","AP Pflanze"]:
            continue
        if column_name.startswith('AP') and not pd.isna(value):
            ap_groups.append(" ".join(column_name.split(" ")[1:]))
        elif column_name.startswith('MF') and not pd.isna(value):
            mf_groups.append(" ".join(column_name.split(" ")[1:]))

    # Add the data to the dictionary
    crop_data = {
        "crop":crop_name,
        "AP": row['AP Pflanze'],
        "AP Groups": ap_groups,
        "MF Groups": mf_groups,
    }
    result[plant_mapping[index]] = crop_data

# Print the resulting dictionary
pp.pprint(result)

{ 0: { 'AP': 4.0,
       'AP Groups': ['BLATT', 'FL', 'L'],
       'MF Groups': [],
       'crop': 'KLEEGRAS'},
  1: { 'AP': 5.0,
       'AP Groups': ['BLATT', 'FL', 'L'],
       'MF Groups': [],
       'crop': 'LUZERNE'},
  2: { 'AP': 3.0,
       'AP Groups': ['BLATT', 'L'],
       'MF Groups': [],
       'crop': 'ACKERBOHNE'},
  3: { 'AP': 5.0,
       'AP Groups': ['BLATT', 'L'],
       'MF Groups': [],
       'crop': 'KÖRNERERBSE'},
  4: { 'AP': 3.0,
       'AP Groups': ['BLATT', 'L'],
       'MF Groups': [],
       'crop': 'SOJABOHNE'},
  5: { 'AP': 4.0,
       'AP Groups': ['BLATT', 'L'],
       'MF Groups': [],
       'crop': 'FUTTERLUPINE'},
  6: { 'AP': 2.0,
       'AP Groups': ['Weizen/Dinkel/Triticale', 'Weizen'],
       'MF Groups': ['GE', 'GE (ohne Mais/Hafer/Hirse)', 'Weizen/Triticale'],
       'crop': 'WINTERWEIZEN'},
  7: { 'AP': 2.0,
       'AP Groups': ['Weizen/Dinkel/Triticale', 'Weizen'],
       'MF Groups': ['GE', 'GE (ohne Mais/Hafer/Hirse)', 'Weizen/Triticale'],
 